In [1]:
import os
from deep_utils import warmup_cosine
from datasets import load_dataset, Audio
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [61]:
help(AutoFeatureExtractor)

Help on class AutoFeatureExtractor in module transformers.models.auto.feature_extraction_auto:

class AutoFeatureExtractor(builtins.object)
 |  This is a generic feature extractor class that will be instantiated as one of the feature extractor classes of the
 |  library when created with the [`AutoFeatureExtractor.from_pretrained`] class method.
 |  
 |  This class cannot be instantiated directly using `__init__()` (throws an error).
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  from_pretrained(pretrained_model_name_or_path, **kwargs) from builtins.type
 |      Instantiate one of the feature extractor classes of the library from a pretrained model vocabulary.
 |      
 |      The feature extractor class to instantiate is selected based on the `model_type` property of the config object
 | 

In [62]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
feature_extractor

loading feature extractor configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/preprocessor_config.json from cache at /home/ai/.cache/huggingface/transformers/d4583dd9e59eb6295f8fe8b18833ae54d963a122d69aa1df7ecce6caafe18c8f.bc3155ca0bae3a39fc37fc6d64829c6a765f46480894658bb21c08db6155358d
loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /home/ai/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/site-packages/transformers/configuration_utils.py:358: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.war

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [39]:
dataset = load_dataset('csv', data_files={'train': "/home/ai/projects/speech/audio_classification/train_gender.csv", 'test': '/home/ai/projects/speech/audio_classification/test_gender.csv'})
dataset = dataset.cast_column("audio_path", Audio(sampling_rate=16_000))
dataset["train"][0]

Using custom data configuration default-1923df7e56bf49ff


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/ai/.cache/huggingface/datasets/csv/default-1923df7e56bf49ff/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'audio_path': {'path': '/home/ai/projects/speech/dataset/irancel-voice-dataset/samples_01/all_segments_final/1024517_000_00_S00_F.wav',
  'array': array([ 6.4552e-05,  0.00048871,  0.00091218, ...,    0.040528,    0.040409,    0.039917], dtype=float32),
  'sampling_rate': 16000},
 'label': 'female'}

In [44]:
import random
import IPython.display as ipd
import librosa
index = random.randint(0, len(dataset['train']))

path = dataset['train'][index]['audio_path']['path']
waveform, sr = librosa.load(path)
text = dataset['train'][index]['label']
print(text)
ipd.Audio(waveform, rate=sr, autoplay=True)

female


In [41]:
labels = set(dataset["train"]['label'])
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
label2id

{'female': '0', 'male': '1'}

In [42]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio_path"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    label = [int(label2id[x]) for x in examples["label"]]
    inputs["label"] = label
    return inputs

In [43]:
encoded_dataset = dataset.map(preprocess_function, remove_columns="audio_path", batched=True)
encoded_dataset['train'][0]

  0%|          | 0/3 [00:00<?, ?ba/s]

Exception ignored in: <function Wave_write.__del__ at 0x7f46c89b71f0>
Traceback (most recent call last):
  File "/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/wave.py", line 326, in __del__
    self.close()
  File "/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/wave.py", line 444, in close
    self._ensure_header_written(0)
  File "/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/wave.py", line 464, in _ensure_header_written
    raise Error('sample width not specified')
wave.Error: sample width not specified
/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/site-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/1 [00:00<?, ?ba/s]

{'label': 0,
 'input_values': [0.39005303382873535,
  0.4227670133113861,
  0.45542803406715393,
  0.4477715492248535,
  0.40922147035598755,
  0.3873559534549713,
  0.40922147035598755,
  0.4477715492248535,
  0.45542803406715393,
  0.4227670133113861,
  0.39005303382873535,
  0.3977324068546295,
  0.436303973197937,
  0.4581150412559509,
  0.436303973197937,
  0.3977324068546295,
  0.39005303382873535,
  0.4227670133113861,
  0.45542803406715393,
  0.4477715492248535,
  0.40922147035598755,
  0.3873559534549713,
  0.40922147035598755,
  0.4477715492248535,
  0.45542803406715393,
  0.4227670133113861,
  0.39005303382873535,
  0.3977324068546295,
  0.436303973197937,
  0.4581150412559509,
  0.436303973197937,
  0.3977324068546295,
  0.39005303382873535,
  0.4227670133113861,
  0.45542803406715393,
  0.4477715492248535,
  0.40922147035598755,
  0.3873559534549713,
  0.40922147035598755,
  0.4477715492248535,
  0.45542803406715393,
  0.4227670133113861,
  0.39005303382873535,
  0.3977324

In [45]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")
    precision = precision_score(labels, predictions, average="weighted")

    return {"accuracy": acc, "f1-score": f1, "recall-score": recall, "precision-score": precision}

In [46]:
import math
import torch
from transformers import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(early_stopping_patience=5)

train_bs = 32 
epochs = 25
lr = 5e-5
lrf = lr
output_dir = "./results"
total_steps = int((np.ceil(encoded_dataset["train"].num_rows / train_bs) * epochs))

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=epochs,
    report_to="tensorboard",
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model='loss',
    per_device_train_batch_size = train_bs,
    per_device_eval_batch_size = 64,
    logging_steps=1,
)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, warmup_cosine(total_steps//10,
                                                                       max_lr=lr,
                                                                       total_steps=total_steps,
                                                                       optimizer_lr=lr,
                                                                       min_lr=1e-6))
# reduce lr with a cosine annealing if total_steps is set to total_steps
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
#     optimizers=(optimizer, scheduler)
)

trainer.train()
trainer.save_model(os.path.join(output_dir, "best"))

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /home/ai/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/site-packages/transformers/configuration_utils.py:358: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1

Epoch,Training Loss,Validation Loss,Accuracy,F1-score,Recall-score,Precision-score
1,0.412900,0.394654,0.864979,0.802356,0.864979,0.748189
2,0.235700,0.321732,0.864979,0.802356,0.864979,0.748189
3,0.602100,0.415825,0.864979,0.802356,0.864979,0.748189
4,0.827000,0.395837,0.864979,0.802356,0.864979,0.748189
5,0.379300,0.389818,0.870605,0.817930,0.870605,0.865847
6,0.597800,0.396064,0.864979,0.802356,0.864979,0.748189
7,0.466800,0.400403,0.864979,0.805017,0.864979,0.816590
8,0.286800,0.299796,0.905767,0.897853,0.905767,0.897000
9,0.219900,0.372356,0.879044,0.839538,0.879044,0.871384
10,0.136400,0.345020,0.891702,0.864202,0.891702,0.887682


***** Running Evaluation *****
  Num examples = 711
  Batch size = 64
/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./results/checkpoint-89
Configuration saved in ./results/checkpoint-89/config.json
Model weights saved in ./results/checkpoint-89/pytorch_model.bin
Feature extractor saved in ./results/checkpoint-89/preprocessor_config.json
Deleting older checkpoint [results/checkpoint-1335] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 711
  Batch size = 64
/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samp

Model weights saved in ./results/checkpoint-1424/pytorch_model.bin
Feature extractor saved in ./results/checkpoint-1424/preprocessor_config.json
Deleting older checkpoint [results/checkpoint-1335] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 711
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1513
Configuration saved in ./results/checkpoint-1513/config.json
Model weights saved in ./results/checkpoint-1513/pytorch_model.bin
Feature extractor saved in ./results/checkpoint-1513/preprocessor_config.json
Deleting older checkpoint [results/checkpoint-1424] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 711
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1602
Configuration saved in ./results/checkpoint-1602/config.json
Model weights saved in ./results/checkpoint-1602/pytorch_model.bin
Feature extractor saved in ./results/checkpoint-1602/preprocessor_config.json
Deleting older checkpoint [results/

In [48]:
import torchaudio
import torch
import librosa
device = "cpu"
model = model.to(device)
waveform, sr = librosa.load("../audio_samples/man_02.mp4")
waveform = torch.from_numpy(waveform).unsqueeze(0)
waveform = torchaudio.transforms.Resample(sr, 16_000)(waveform)
inputs = feature_extractor(waveform, sampling_rate=feature_extractor.sampling_rate,
                           max_length=16000, truncation=True)
tensor = torch.tensor(inputs['input_values'][0]).to(device)
with torch.no_grad():
    output = model(tensor)
    logits = output['logits'][0]
    label_id = torch.argmax(logits).item()
label_name = id2label[str(label_id)]
print(label_name)

/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


male
